# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [1]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

In [3]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
]

In [4]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [5]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [6]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [7]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [8]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [9]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题']


In [10]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题


In [11]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [12]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题

<< INPUT >>
{input}

<< OUTPUT (must include ```json at the start of the

In [13]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [14]:
print(chain.run("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...


/home/ubuntu/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


物理: {'input': '黑体辐射是什么？'}
> Finished chain.


黑体辐射是指一个理想的物体，它可以吸收所有进入它内部的电磁辐射，并完全发射出来。它的辐射性质与它的温度有关，随着温度的升高，它会发出更多的辐射，也会发出更短的波长的辐射。这个理想的概念在物理学领域中被广泛应用，可以用来解释许多物理现象，如热辐射和宇宙微波背景辐射。


In [15]:
print(
    chain.run(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.


首先，我们需要找到大于40的第一个质数。质数是只能被1和自身整除的数，所以我们可以从41开始，依次往后找，直到找到一个质数为止。

41是一个质数，但是加一后是42，不能被3整除，所以我们继续往后找。

43是一个质数，加一后是44，也不能被3整除，继续往后找。

47是一个质数，加一后是48，可以被3整除，所以大于40的第一个质数是47。

因此，47加一后为48，满足条件的质数为48÷3=16。

所以，大于40的第一个质数是47，使得这个质数加一后能被3整除的数是16。


In [16]:
router_chain.verbose = True

In [17]:
print(chain.run("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.


黑洞是宇宙中一种非常密集的天体，其质量非常巨大，但体积非常小。它的引力非常强大，甚至连光也无法逃离它的吸引力。这是因为黑洞的质量曲折了时空，使光线弯曲，从而让它看起来像一个黑暗的球体。黑洞的形成是因为某些恒星在死亡后，其质量会塌缩到极限，形成一个无法逃脱的引力陷阱。我们目前对黑洞的了解仍然不够完整，但它们在宇宙中扮演着重要的角色，影响着星系的演化。希望我的回答能够解决您的疑问。


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [1]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

biology_template = """你是一位经验丰富的生物学家。
你精通生物学的所有分支，从分子生物学、遗传学到生态学和进化论。
你善于以直观且易于理解的方式解释复杂的生物概念。
当你遇到自己知识范围外的问题时，你会诚实告知，并引导提问者寻找答案的途径。

这是一个问题：
{input}"""

cs_template = """你是一位资深的计算机科学家。
无论是编程语言、算法、数据结构还是人工智能，你都能游刃有余。
你有能力将复杂的技术问题简化，让非专业人士也能理解。
面对未知挑战，你勇于探索，乐于分享你的发现。

这是一个问题：
{input}"""

chinese_language_literature_template = """你是一位博学的汉语言文学教授。
你对汉字、古文、诗词歌赋及中国古典文学有着深厚的造诣。
你能够深入浅出地解析文学作品，揭示其深层的文化和历史背景。
面对不熟悉的领域，你保持谦逊，鼓励持续学习和探索。

这是一个问题：
{input}"""


prompt_infos = [
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": cs_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_language_literature_template,
    },
]

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

D:\workspace\solfware\code\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [3]:
print("--------生物问题--------")
print(chain.run("DNA是由什么组成的？"))
print("--------计算机问题--------")
print(chain.run("人工智能的底层算法是什么？"))
print("--------汉语言问题--------")
print(chain.run("中文的发展历程？"))

--------生物问题--------


> Entering new MultiPromptChain chain...
生物: {'input': 'DNA是由什么组成的？'}
> Finished chain.

DNA由四种碱基（腺嘌呤、鸟嘌呤、胞嘧啶、鸟嘌呤）组成，以及磷酸和脱氧核糖组成的糖磷酸骨架。碱基按照一定的顺序排列形成DNA的双螺旋结构。
--------计算机问题--------


> Entering new MultiPromptChain chain...
计算机: {'input': '人工智能的底层算法是什么？'}
> Finished chain.


人工智能的底层算法是指用于解决人工智能问题的基本算法，如机器学习算法、神经网络算法、遗传算法等。这些算法可以帮助计算机从数据中学习、理解和推断，从而实现智能化的功能。
--------汉语言问题--------


> Entering new MultiPromptChain chain...
汉语言文学: {'input': '中文的发展历程？'}
> Finished chain.


中文的发展历程可以追溯到约5000年前的甲骨文，随后演变为金文、篆文、隶书等，最终形成今天所使用的楷书。随着历史的发展，中文也经历了古代汉语、中古汉语、近代汉语等不同发展阶段，同时受到其他语言和文化的影响，如佛教文化、外来民族的语言等。在现代，随着科技的进步和社会的变化，中文也不断发展、演变，出现了许多新的词汇和表达方式。总的来说，中文的发展历程是一个与中国历史和文化密切相关的过程，不断吸收、发展和创新，反映着中国人民的思想和生活。
